In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance

In [2]:
df = pd.read_csv('data.csv')

In [ ]:
z = np.random.normal(0, 1, size=X_new_ori.shape)
X_new = X_new_ori + z

In [ ]:
df

In [3]:
train, test = train_test_split(df, test_size = 0.2, random_state = 42)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [54]:
train.describe()

,Bp,Sg,Al,Su,Rbc,Bu,Sc,Sod,Pot,Hemo,Wbcc,Rbcc,Htn,Class
count,320.000000,320.000000,320.00000,320.000000,320.00000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000
mean,75.956250,1.017750,0.99375,0.384375,0.89375,55.386563,2.873625,137.464750,4.680781,12.589656,8422.937500,4.719031,0.352313,0.618750
std,13.833242,0.005494,1.25694,1.019745,0.30864,48.875895,5.424476,9.778089,3.125634,2.662750,2563.656577,0.815552,0.476909,0.486454
min,50.000000,1.005000,0.00000,0.000000,0.00000,10.000000,0.400000,4.500000,2.500000,3.100000,2200.000000,2.100000,0.000000,0.000000
25%,70.000000,1.015000,0.00000,0.000000,1.00000,27.000000,0.900000,135.000000,4.000000,10.900000,6900.000000,4.500000,0.000000,0.000000
50%,76.000000,1.020000,0.50000,0.000000,1.00000,42.000000,1.350000,137.530000,4.630000,12.530000,8406.000000,4.710000,0.000000,1.000000
75%,80.000000,1.020000,2.00000,0.000000,1.00000,57.000000,2.925000,140.250000,4.800000,14.700000,9400.000000,5.100000,1.000000,1.000000
max,180.000000,1.025000,5.00000,5.000000,1.00000,391.000000,76.000000,163.000000,47.000000,17.800000,26400.000000,8.000000,1.000000,1.000000


In [4]:
y_train = train['Class']
X_train = train.loc[:, train.columns!='Class']
y_test = test['Class']
X_test = test.loc[:, test.columns!='Class']

In [91]:
X_new2 = pd.DataFrame(X_new)

In [92]:
lgt = LogisticRegression(random_state=42, penalty='none')
lgt.fit(X_new2, y_train)
np.mean(lgt.predict(X_test_new) == y_test)

0.8

In [5]:
def dropout(algo, X, Y, M, K, criteria):
    """
    This function uses dropout method to train a predicted model that optimizes the specific criterion.
    @param algo(required): A learning algorithm that takes X, Y as inputs and outputs a model 
    @param X(required): The matrix of indepdent/predictor variables
    @param Y(required): The outcome variable
    @param M(required): The number of Monte Carlo replicates
    @param K(required): Number of CV folds for tuning hyperparameter
    @param criteria(required): A criterion to be used to evaluate the method. 
    Can be either mean squared error (MSE) or mean absolute deviation (MAD)
    @return: A predictive model that optimizes the specific criterion
    """
    
    # List of phi values to be tuned
    phi_list = np.arange(0, 1, 0.1)
    cv_error = []
    
    for phi in phi_list:
        # Create k folds for cross-validation
        kf = KFold(n_splits = K, shuffle=True)
        error = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            
            # Monte Carlo: expand original data M times to generate a new dataset 
            # (X will be modified later)
            X_new_ori = np.repeat(X_train, M, axis=0)
            Y_new = np.repeat(Y_train, M, axis=0)
            
            # Generate independent Bernoulli random variables Z
            z = np.random.binomial(1, phi, size=X_new_ori.shape)
            # Random Dropout for X
            X_new = X_new_ori*z/(1-phi)
            
            # Fit input learning algorithm with new data
            reg = algo(X_new, Y_new)
            # Predict Y using X test
            pred = reg.predict(X_test)
            
            # Calculate MSE/MAD using the model obtained above
            if criteria == "MSE":
                error.append(mean_squared_error(Y_test, pred))
            elif criteria == "MAD":
                error.append(mean_absolute_error(Y_test, pred))
            # Raise error if input is not MSE or MAD
            else:
                raise ValueError('Please input either MSE or MAD!')
        
        cv_error.append(np.mean(error))
    
    # Find optimal phi value that has the smallest CV error
    phi_opt = phi_list[np.argmin(cv_error)]
    
    # Use the optimal phi value to train the predicted model
    X_new_ori = np.repeat(X, M, axis=0)
    Y_new = np.repeat(Y, M, axis=0)
    
    z = np.random.binomial(1, phi_opt, size=X_new_ori.shape)
    X_new = X_new_ori*z/(1-phi_opt)

    reg = algo(X_new, Y_new)
    
    return reg

In [62]:
def lgt_drop(X, Y):
    return LogisticRegression(random_state=42, penalty='none').fit(X, Y)

In [63]:
X = X_train.values
y = y_train.values

In [64]:
z = np.random.normal(0, 3, size=X.shape)
X_new = X + z

In [65]:
z2 = np.random.normal(0, 20, size=X_test.shape)
X_test_new = X_test + z2

In [66]:
lgt_drop_fit = dropout(lgt_drop, X_new, y, 100, 5, 'MSE')

In [67]:
np.mean(lgt_drop_fit.predict(X_test_new) == y_test)

0.6375

In [99]:
lgt_dropup_fit = dropout_updated(lgt_drop, X_new, y, 100, 5, 'MSE')

In [100]:
np.mean(lgt_dropup_fit.predict(X_test_new) == y_test)

0.65

In [98]:
def dropout_updated(algo, X, Y, M, K, criteria):
    """
    This function uses dropout method to train a predicted model that optimizes the specific criterion.
    @param algo(required): A learning algorithm that takes X, Y as inputs and outputs a model 
    @param X(required): The matrix of indepdent/predictor variables
    @param Y(required): The outcome variable
    @param M(required): The number of Monte Carlo replicates
    @param K(required): Number of CV folds for tuning hyperparameter
    @param criteria(required): A criterion to be used to evaluate the method. 
    Can be either mean squared error (MSE) or mean absolute deviation (MAD)
    @return: A predictive model that optimizes the specific criterion
    """
    
    # List of phi values to be tuned
    cv_error = []
    # Create k folds for cross-validation
    kf = KFold(n_splits = K, shuffle=True)
    error = []
    p_up_list = np.arange(0.5, 1, 0.1)
    
    for p_up in p_up_list:
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]

            # Monte Carlo: expand original data M times to generate a new dataset 
            # (X will be modified later)
            X_new_ori = np.repeat(X_train, M, axis=0)
            Y_new = np.repeat(Y_train, M, axis=0)

            # Generate independent Bernoulli random variables Z
            center = np.mean(X_new_ori, axis=0)
            dist = [distance.euclidean(point, center) for point in X_new_ori]
            quant_list = np.quantile(dist, [0, 0.25, 0.5, 0.75, 1])
            dist_df = pd.DataFrame({'dist': dist})
            dist_df['category'] = pd.cut(dist_df['dist'], quant_list, 
                                         labels=['0-25%', '25-50%', '50-75%', '75-100%'])
            dist_df['prob'] = 0
            dist_df.loc[dist_df.category == '0-25%', 'prob'] = p_up
            dist_df.loc[dist_df.category == '25-50%', 'prob'] = p_up - 0.1
            dist_df.loc[dist_df.category == '50-75%', 'prob'] = p_up - 0.2
            dist_df.loc[dist_df.category == '75-100%', 'prob'] = p_up - 0.3
            phi_list = dist_df['prob'].values
            phi_list = phi_list.reshape(-1,1)
            z_list = np.random.binomial(1, phi_list, size=X_new_ori.shape)

            # Random Dropout for X
            X_new = X_new_ori*z_list/(1-phi_list)

            # Fit input learning algorithm with new data
            reg = algo(X_new, Y_new)
            # Predict Y using X test
            pred = reg.predict(X_test)

            # Calculate MSE/MAD using the model obtained above
            if criteria == "MSE":
                error.append(mean_squared_error(Y_test, pred))
            elif criteria == "MAD":
                error.append(mean_absolute_error(Y_test, pred))
            # Raise error if input is not MSE or MAD
            else:
                raise ValueError('Please input either MSE or MAD!')

        cv_error.append(np.mean(error))
    
    # Find optimal p_up value that has the smallest CV error
    p_up_opt = p_up_list[np.argmin(cv_error)]
    
    # Use the optimal phi value to train the predicted model
    X_new_ori = np.repeat(X, M, axis=0)
    Y_new = np.repeat(Y, M, axis=0)
    
    # Generate independent Bernoulli random variables Z
    center = np.mean(X_new_ori, axis=0)
    dist = [distance.euclidean(point, center) for point in X_new_ori]
    quant_list = np.quantile(dist, [0, 0.25, 0.5, 0.75, 1])
    dist_df = pd.DataFrame({'dist': dist})
    dist_df['category'] = pd.cut(dist_df['dist'], quant_list, 
                                 labels=['0-25%', '25-50%', '50-75%', '75-100%'])
    dist_df['prob'] = 0
    dist_df.loc[dist_df.category == '0-25%', 'prob'] = p_up_opt
    dist_df.loc[dist_df.category == '25-50%', 'prob'] = p_up_opt - 0.1
    dist_df.loc[dist_df.category == '50-75%', 'prob'] = p_up_opt - 0.2
    dist_df.loc[dist_df.category == '75-100%', 'prob'] = p_up_opt - 0.3
    phi_list = dist_df['prob'].values
    phi_list = phi_list.reshape(-1,1)
    z_list = np.random.binomial(1, phi_list, size=X_new_ori.shape)

    # Random Dropout for X
    X_new = X_new_ori*z_list/(1-phi_list)

    # Fit input learning algorithm with new data
    reg = algo(X_new, Y_new)
    
    return reg